In [12]:
import pandas as pd
import os
import pickle
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle


## Train Static Valance & Arousal Using Linear SVR

In [2]:
DATASET_DIR = 'PMEmo/dataset/PMEmo2019'

def load_static_features():
    features_csv = os.path.join(DATASET_DIR, 'features', 'static_features.csv')
    static_features= pd.read_csv(features_csv, index_col=0)
    return static_features

def load_static_features_and_valence():
    static_features = load_static_features()
    valence_csv = os.path.join(DATASET_DIR, 'annotations', 'static_annotations.csv')
    valence = pd.read_csv(valence_csv, index_col=0, usecols=['musicId','Valence(mean)'])
    return static_features.join(valence).dropna()

def load_static_features_and_arousal():
    static_features = load_static_features()
    arousal_csv = os.path.join(DATASET_DIR, 'annotations', 'static_annotations.csv')
    arousal = pd.read_csv(arousal_csv, index_col=0, usecols=['musicId','Arousal(mean)'])
    return static_features.join(arousal).dropna()

def load_audio_dataset(data):
    features = data[data.columns[:-1]].values
    labels = data[data.columns[-1]].values
#     scaler = StandardScaler(copy=False)
#     scaler.fit_transform(features)
    return features, labels

In [3]:
data_static_a = load_static_features_and_arousal()
static_features, static_labels_arousal = load_audio_dataset(data_static_a)

data_static_v = load_static_features_and_valence()
static_features, static_labels_valence = load_audio_dataset(data_static_v)

In [4]:
scaler = StandardScaler().fit(static_features)
scaled_static_features = scaler.transform(static_features)

In [5]:
static_arousal_model = SVR(kernel='linear', gamma='scale')
static_arousal_model.fit(scaled_static_features, static_labels_arousal)
with open('svr_linear_static_arousal.pkl', 'wb') as fid:
    pickle.dump(static_arousal_model, fid)

In [6]:

static_valence_model = SVR(kernel='linear', gamma='scale')
static_valence_model.fit(scaled_static_features, static_labels_valence)
with open('svr_linear_static_valence.pkl', 'wb') as fid:
    pickle.dump(static_valence_model, fid)

## Train Dynamic Valance & Arousal Using RBF SVR

In [10]:
features = pd.read_csv(os.path.join(DATASET_DIR, 'features', 'dynamic_features.csv'))
annotations = pd.read_csv(os.path.join(DATASET_DIR, 'annotations', 'dynamic_annotations.csv'))
dataset = pd.merge(features, annotations, on=['musicId', 'frameTime'])


In [13]:
songs = dataset['musicId'].unique()
songs = shuffle(songs, random_state=3)
test_num = 0
testsongs = songs[:test_num]

In [14]:

dynamic_features = dataset[dataset.columns[2:262]]
dynamic_scaler = StandardScaler().fit(dynamic_features)
scaled_dynamic_features = dynamic_scaler.transform(dynamic_features)

dynamic_labels_arousal, dynamic_labels_valence = dataset['Arousal(mean)'], dataset['Valence(mean)']

In [15]:
scaled_dynamic_features.shape, dynamic_labels_valence.shape, dynamic_labels_arousal.shape

((36434, 260), (36434,), (36434,))

In [16]:
dynamic_arousal_model = SVR(kernel='rbf', gamma='scale')
dynamic_arousal_model.fit(scaled_dynamic_features, dynamic_labels_arousal)

with open('svr_rbf_dynamic_arousal.pkl' , 'wb') as fid:
        pickle.dump(dynamic_arousal_model, fid)
   


In [17]:
dynamic_valence_model = SVR(kernel='rbf', gamma='scale')
dynamic_valence_model.fit(scaled_dynamic_features, dynamic_labels_valence)

with open('svr_rbf_dynamic_valence.pkl' , 'wb') as fid:
        pickle.dump(dynamic_valence_model, fid)